# Gen_PDB_ROTAMERS

导入所需要的库

In [1]:
import os
import argparse
import re
import numpy as np
import shutil
from rdkit.Chem import Lipinski
from Bio.PDB import PDBParser, PDBIO, Superimposer
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdForceFieldHelpers import UFFGetMoleculeForceField
import random

读入SMILES文件及其他参数

In [12]:
'''
以下参数在脚本实际运行中为argparse参数,此处为了便于调试将其内置
-i: SMILES文件, 需要和参数化时使用的SMILES保持一致
-n: NCAA三字母缩写, 需要和参数化时使用的NCAA名保持一致
-m: 参考文件, 用于进行原子指认, 为参数化时得到的Rosetta临时文件
-d: RMSD阈值, 用于判断是否需要进行能量最小化, 默认值为0.1, 在不进行任何认为设置时会根据氨基酸chi角数量自动调整该阈值大小, 若人为干预则将不会进行自动调整, 而是采用用户输入的值
-c: cut_off值，默认根据chi角数量自动调整该值大小，若人为干预则将不会进行自动调整, 而是采用用户输入的值
'''

args_input='input.smiles'
args_names=['OAS']
args_params='OAS_temps.params'
args_rmsd_threshold=0.1
args_cut_off=10000

SMARTS反应定义

In [13]:
# 定义乙酰化和甲氨基化的SMARTS模式
acetylation_smarts = '[N:1][C:2][C:3](=[O:4])>>[C:5][C:7](=[O:6])[N:1][C:2][C:3](=[O:4])'
amidation_smarts = '[C:1][C:2](=[O:3])[O:4]>>[C:1][C:2]([N:5][C:6])(=[O:3])'

计算chi角数量并定义cut_off值

In [14]:
def count_rotatable_bonds(smiles_file,cut_off):
    with open(smiles_file, 'r') as file:
        smiles_list = [line.strip() for line in file.readlines()]

    results = []

    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"SMILES字符串 '{smiles}' 无法解析")
            continue

        #计算自由旋转角个数，由于其包含主链上的一个自由旋转角，因此chi角个数为自由旋转角个数-1
        #c_value表示初始构象数，v_value表示cut_off值，如果用户通过参数干预，则v_value将会被设置为用户输入的值
        num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
        if num_rotatable_bonds==2 and cut_off==10000:
            v_value=10
            c_value=10000
        elif num_rotatable_bonds==3 and cut_off==10000:
            v_value=30
            c_value=10000
        elif num_rotatable_bonds==4 and cut_off==10000:
            v_value=90
            c_value=10000
        elif num_rotatable_bonds==5 and cut_off==10000:
            v_value=270
            c_value=10000
        elif num_rotatable_bonds>=6 and cut_off==10000:
            v_value=810
            c_value=10000
        else:
            v_value=cut_off
            c_value=10000
        results.append((smiles, num_rotatable_bonds, c_value, v_value))

    return results

results = count_rotatable_bonds(args_input,args_cut_off)
for smiles, num_rotatable_bonds, c_value, v_value in results:
    print(f"SMILES: {smiles}  自由旋转键数目: {num_rotatable_bonds}  c_value: {c_value}  v_value: {v_value}")
    # 将c_value和v_value输出为全局变量
    globals()['c_value'] = c_value
    globals()['v_value'] = v_value


SMILES: N[C@H](C(O)=O)COC(C)=O  自由旋转键数目: 3  c_value: 10000  v_value: 1000


生成初始构象库

In [15]:
#为NCAA进行封端，并输出标准化的SMILES字符串
def process_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError(f"Unable to parse SMILES: '{smiles}'")

    # 创建乙酰化和甲氨基化的反应
    rxn_acetylation = AllChem.ReactionFromSmarts(acetylation_smarts)
    rxn_amidation = AllChem.ReactionFromSmarts(amidation_smarts)

    # 应用甲氨基化反应
    products_amidation = rxn_amidation.RunReactants((mol,))
    if not products_amidation:
        raise ValueError("C-terminal amidation failed")

    product_amidation = products_amidation[0][0]

    # 应用乙酰化反应
    products_acetylation = rxn_acetylation.RunReactants((product_amidation,))
    if not products_acetylation:
        raise ValueError("N-terminal acetylation failed")

    product_acetylation = products_acetylation[0][0]

    # 获取乙酰化后的分子的 SMILES
    final_smiles = Chem.MolToSmiles(product_acetylation, canonical=False)
    return final_smiles

# 将SMILES字符串转换为PDB文件
def smiles_to_pdb(smiles_list, names_list, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if len(names_list) < len(smiles_list):
        names_list.extend(['UAA'] * (len(smiles_list) - len(names_list)))
    elif len(names_list) > len(smiles_list):
        raise ValueError("The number of provided names exceeds the number of SMILES strings.")

    generated_files = []
    for i, (smiles, name) in enumerate(zip(smiles_list, names_list)):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"SMILES字符串 '{smiles}' 无法解析")
            continue
        mol = Chem.AddHs(mol)
        # 设置随机种子以确保每次生成的构象具有更大的差异性
        random_seed = random.randint(1, 1000000)
        params = AllChem.ETKDG()
        params.randomSeed = random_seed
        AllChem.EmbedMolecule(mol, params)
        
        # 使用UFF力场进行结构优化
        ff = UFFGetMoleculeForceField(mol)
        ff.Initialize()
        ff.Minimize()

        # 获取结构优化后的能量值
        energy = ff.CalcEnergy()

        # 生成PDB文件名
        pdb_filename = os.path.join(output_dir, f'{name}_{i+1}.pdb')
        
        # 写入PDB文件并添加能量和SMILES信息
        with open(pdb_filename, 'w') as pdb_file:
            pdb_file.write(f"REMARK SMILES: {smiles}\n")
            pdb_file.write(f"REMARK Energy after optimization: {energy:.2f} kcal/mol\n")
            pdb_file.write(Chem.rdmolfiles.MolToPDBBlock(mol))
        
        generated_files.append(pdb_filename)
        print(f"生成 {pdb_filename}")
        

    return generated_files

# 调试用，便于为函数传参
input_file=args_input
names_list=args_names
combined_file = 'combined_pdb_files.pdb'
print(input_file)
print(names_list)

# 生成封端后的SMILES字符串
def gen_smiles(input_file, names_list):

    with open(input_file, 'r') as f:
        smiles_list = [line.strip() for line in f.readlines()]

    if len(names_list) != len(smiles_list):
        raise ValueError("The number of provided names does not match the number of SMILES strings.")

    processed_smiles_list = []
    for smiles in smiles_list:
        try:

            processed_smiles = process_smiles(smiles)
            processed_smiles_list.append(processed_smiles)
        except ValueError as e:
            print(e)
            continue
    return processed_smiles_list

# 将smiles转化为pdb
def gen_conformers(processed_smiles_list, names_list):

    output_dir = 'pdb_files'
    generated_files = smiles_to_pdb(processed_smiles_list, names_list, output_dir)

# 将每一轮循环中生成的pdb文件追加到combined_pdb_files.pdb中
def append_pdb_to_combined_file(pdb_file, combined_file):
    with open(pdb_file, 'r') as f_pdb:
        pdb_content = f_pdb.read()
    
    with open(combined_file, 'a') as f_combined:
        f_combined.write(pdb_content)

if os.path.exists(combined_file):
    os.remove(combined_file)

# 生成封端后的SMILES
processed_smiles_list = gen_smiles(input_file, names_list)

# 循环调用gen_conformers函数，直到生成c_value个构象(10000个)
for i in range(c_value):
    print(f"Running iteration {i + 1}...")
    gen_conformers(processed_smiles_list, names_list)

    # 将pdb_files文件夹中的所有PDB文件内容追加到combined_pdb_files.pdb中
    for pdb_file in os.listdir('pdb_files'):
        if pdb_file.endswith('.pdb'):
            pdb_filepath = os.path.join('pdb_files', pdb_file)
            append_pdb_to_combined_file(pdb_filepath, combined_file)
    
    print(f"Iteration {i + 1} completed.")

input.smiles
['OAS']
Running iteration 1...
生成 pdb_files/OAS_1.pdb
Iteration 1 completed.
Running iteration 2...
生成 pdb_files/OAS_1.pdb
Iteration 2 completed.
Running iteration 3...
生成 pdb_files/OAS_1.pdb
Iteration 3 completed.
Running iteration 4...
生成 pdb_files/OAS_1.pdb
Iteration 4 completed.
Running iteration 5...
生成 pdb_files/OAS_1.pdb
Iteration 5 completed.
Running iteration 6...
生成 pdb_files/OAS_1.pdb
Iteration 6 completed.
Running iteration 7...


[15:41:17] product atom-mapping number 5 not found in reactants.
[15:41:17] product atom-mapping number 6 not found in reactants.
[15:41:17] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 4 
[15:41:17] product atom-mapping number 5 not found in reactants.
[15:41:17] product atom-mapping number 7 not found in reactants.
[15:41:17] product atom-mapping number 6 not found in reactants.


生成 pdb_files/OAS_1.pdb
Iteration 7 completed.
Running iteration 8...
生成 pdb_files/OAS_1.pdb
Iteration 8 completed.
Running iteration 9...
生成 pdb_files/OAS_1.pdb
Iteration 9 completed.
Running iteration 10...
生成 pdb_files/OAS_1.pdb
Iteration 10 completed.
Running iteration 11...
生成 pdb_files/OAS_1.pdb
Iteration 11 completed.
Running iteration 12...
生成 pdb_files/OAS_1.pdb
Iteration 12 completed.
Running iteration 13...
生成 pdb_files/OAS_1.pdb
Iteration 13 completed.
Running iteration 14...
生成 pdb_files/OAS_1.pdb
Iteration 14 completed.
Running iteration 15...
生成 pdb_files/OAS_1.pdb
Iteration 15 completed.
Running iteration 16...
生成 pdb_files/OAS_1.pdb
Iteration 16 completed.
Running iteration 17...
生成 pdb_files/OAS_1.pdb
Iteration 17 completed.
Running iteration 18...
生成 pdb_files/OAS_1.pdb
Iteration 18 completed.
Running iteration 19...
生成 pdb_files/OAS_1.pdb
Iteration 19 completed.
Running iteration 20...
生成 pdb_files/OAS_1.pdb
Iteration 20 completed.
Running iteration 21...
生成 pdb_file

按能量从小到大对combined_pdb_file.pdb进行排序，以保证低能量构象被优先存入PDB ROTAMERS库中

In [6]:
n_value = args_names[0]
combined_pdb_file = 'combined_pdb_files.pdb'  # 合并的PDB文件名
output_directory = 'split_pdb_files'  # 存放拆分后PDB文件的目录

# 根据combined_pdb_file中的能量注释行提取能量值
def extract_energy_from_pdb(pdb_content):
    lines = pdb_content.splitlines()
    energy = None
    for line in lines:
        if line.startswith("REMARK Energy after optimization:"):
            energy = float(re.search(r'\d+\.\d+', line).group())
            break
    if energy is None:
        raise ValueError("No energy information found in PDB content")
    return energy

# 对PDB文件中的原子编号进行重新排序
def process_pdb_file(filepath, n_value, output_filepath, energy_value):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # 添加能量值注释行
    energy_line = f"REMARK Energy after optimization: {energy_value:.2f}\n"

    # 过滤掉不是以 "HETATM" 或 "ATOM  " 开头的行
    lines = [line for line in lines if line.startswith("HETATM") or line.startswith("ATOM  ")]

    # 字典存储需要重新排序的行
    line_dict = {"C2": None, "O1": None, "C1": None, "H1": None, "H2": None, "H3": None,
                 "N2": None, "C5": None, "H6": None, "H7": None, "H8": None, "H9": None,
                 "N1": None, "C3": None, "C4": None, "O2": None, "H4": None, "H5": None}
    other_lines = []

    for line in lines:
        atom_name = line[12:16].strip()
        if atom_name in line_dict:
            line_dict[atom_name] = line
        else:
            other_lines.append(line)

    # 按指定顺序重新排列行
    reordered_lines = []
    for key in ["C2", "O1", "C1", "H1", "H2", "H3", "N2", "C5", "H6", "H7", "H8", "H9", "N1", "C3", "C4", "O2"]:
        if line_dict[key] is not None:
            reordered_lines.append(line_dict[key])

    reordered_lines.extend(other_lines)

    if line_dict["H4"] is not None:
        reordered_lines.append(line_dict["H4"])

    if line_dict["H5"] is not None:
        reordered_lines.append(line_dict["H5"])

    # 修改残基名
    for i in range(len(reordered_lines)):
        if i < 6:
            reordered_lines[i] = reordered_lines[i][:17] + "ACE" + reordered_lines[i][20:]
        elif i < 12:
            reordered_lines[i] = reordered_lines[i][:17] + "NME" + reordered_lines[i][20:]
        else:
            reordered_lines[i] = reordered_lines[i][:17] + n_value + reordered_lines[i][20:]

    # 在开头添加能量值注释行
    reordered_lines.insert(0, energy_line)

    # 添加 "END" 行
    reordered_lines.append("END\n")

    # 写回新文件
    with open(output_filepath, 'w') as file:
        file.writelines(reordered_lines)

# 拆分合并的PDB文件并按能量值由小到大排序
def split_combined_pdb(combined_pdb_file, output_dir, n_value):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    pdb_files = []
    energies = {}

    with open(combined_pdb_file, 'r') as f:
        content = f.read().strip().split('END\n')

    for idx, pdb_block in enumerate(content):
        if pdb_block.strip():
            pdb_content = pdb_block.strip() + '\nEND'
            energy = extract_energy_from_pdb(pdb_content)

            # 创建PDB文件名，使用能量值命名
            pdb_filename = os.path.join(output_dir, f'conformer_{energy:.2f}.pdb')

            with open(pdb_filename, 'w') as pdb_out:
                pdb_out.write(pdb_content)

            pdb_files.append((pdb_filename, energy))
            energies[pdb_filename] = energy

            # 对每个拆分的PDB文件的原子编号进行重新排序
            output_filepath = f"{pdb_filename}_processed.pdb"
            process_pdb_file(pdb_filename, n_value, output_filepath, energy)
            print(f"已处理PDB文件: {output_filepath}")

    # 按能量排序生成的PDB文件
    sorted_files = sorted(pdb_files, key=lambda x: x[1])

    # 创建新的合并PDB文件并按能量顺序写入排序后的pdb
    combined_sorted_file = 'combined_sorted_pdb_files.pdb'
    with open(combined_sorted_file, 'w') as f_combined:
        for pdb_file, _ in sorted_files:
            with open(pdb_file + "_processed.pdb", 'r') as f_pdb:
                pdb_content = f_pdb.read()
                f_combined.write(pdb_content)

    return combined_sorted_file

try:
    sorted_combined_file = split_combined_pdb(combined_pdb_file, output_directory, n_value)
    print(f"已创建排序后的合并PDB文件: {sorted_combined_file}")

except Exception as e:
    print(f"错误: {e}")

已处理PDB文件: split_pdb_files/conformer_20.15.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_13.38.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_13.94.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_14.68.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_15.23.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_24.56.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_20.31.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_15.38.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_13.56.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_12.98.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_17.14.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_16.10.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_14.73.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_17.19.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_17.40.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_17.00.pdb_processed.pdb
已处理PDB文件: split_pdb_files/conformer_16.2

In [8]:
rmsd_threshold = args.rmsd_threshold
if rmsd_threshold==0.001 and v_value==10:
    rmsd_threshold = 0.80
elif rmsd_threshold==0.001 and v_value==30:
    rmsd_threshold = 0.60
elif rmsd_threshold==0.001 and v_value==90:
    rmsd_threshold = 0.40
elif rmsd_threshold==0.001 and v_value==270:
    rmsd_threshold = 0.20
elif rmsd_threshold==0.001 and v_value==810:
    rmsd_threshold = 0.10
print(rmsd_threshold)

# 定义主链原子名
backbone_atoms = {'C3', 'C4', 'O1', 'N1', 'O2'}

# 获取主链原子
def get_backbone_atoms(structure):

    return [atom for atom in structure.get_atoms() if atom.get_name() in backbone_atoms]

# 按主链对所有pdb进行align，并使用对齐后的pdb文件并覆盖原始PDB文件
def align_and_overwrite(ref_structure, pdb_file, parser):

    structure = parser.get_structure(pdb_file, pdb_file)
    
    # 获取主链原子
    ref_atoms = get_backbone_atoms(ref_structure)
    mobile_atoms = get_backbone_atoms(structure)
    
    # 使用 Superimposer 进行对齐
    super_imposer = Superimposer()
    super_imposer.set_atoms(ref_atoms, mobile_atoms)
    super_imposer.apply(structure.get_atoms())

    # 保存对齐后的结构到原始PDB文件中
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_file)

# Kabsch算法计算最佳旋转矩阵
def kabsch_algorithm(P, Q):
    # 中心化
    P_centered = P - np.mean(P, axis=0)
    Q_centered = Q - np.mean(Q, axis=0)
    
    # 计算协方差矩阵
    H = np.dot(P_centered.T, Q_centered)
    
    # 计算SVD
    U, S, Vt = np.linalg.svd(H)
    
    # 计算旋转矩阵
    V = Vt.T
    d = np.sign(np.linalg.det(np.dot(V, U.T)))
    R = np.dot(V, np.dot(np.diag([1, 1, d]), U.T))
    
    return R

# 计算两个点集之间的RMSD
def calculate_rmsd(P, Q):

    P_centered = P - np.mean(P, axis=0)
    Q_centered = Q - np.mean(Q, axis=0)
    
    R = kabsch_algorithm(P_centered, Q_centered)
    P_rot = np.dot(P_centered, R)
    
    diff = P_rot - Q_centered
    rmsd = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    
    return rmsd

# 从PDB文件中提取排除指定原子外的坐标和名称
def get_coordinates(structure, atoms_to_exclude):

    atoms = []
    coords = []
    for atom in structure.get_atoms():
        if atom.get_name() not in atoms_to_exclude and 'H' not in atom.get_name():  # 排除主链，封端和所有氢原子
            atoms.append(atom.get_name())
            coords.append(atom.get_coord())
    
    return np.array(coords), atoms

# 计算两个构象之间的RMSD
def align_and_calculate_rmsd(ref_structure, temp_structure, original_pdb_file):
    # 指定在RMSD计算中要排除的主链，封端和所有氢原子
    atoms_to_exclude = {'C1', 'C2', 'C4', 'C5', 'O1', 'O2', 'N1', 'N2'}
    
    coords1, atom_names1 = get_coordinates(ref_structure, atoms_to_exclude)
    coords2, atom_names2 = get_coordinates(temp_structure, atoms_to_exclude)
    
    if coords1.shape != coords2.shape:
        raise ValueError("两个PDB文件的坐标数量不一致")
    
    rmsd_value = calculate_rmsd(coords1, coords2)
    
    return rmsd_value

# 遍历每一个构象，将其与最终构象库中的所有构象进行RMSD比对，当且仅当被遍历的构象与最终构象库中的所有构象的RMSD值均大于阈值时，将其写入最终的合并文件
def align_structures_and_check_rmsd(pdb_files, parser, aligned_combined_pdb_filename, rmsd_threshold):
    aligned_structures = []
    conformer_count = 0

    with open(aligned_combined_pdb_filename, 'w') as aligned_combined_pdb:
        ref_pdb_file = pdb_files[0]
        ref_structure = parser.get_structure('reference', ref_pdb_file)
        align_and_overwrite(ref_structure, ref_pdb_file, parser)
        aligned_structures.append(ref_structure)

        for pdb_file in pdb_files[1:]:
            if conformer_count >=v_value or conformer_count >= len(pdb_files):
                break

            align_and_overwrite(ref_structure, pdb_file, parser)
            temp_structure = parser.get_structure('temp_aligned', pdb_file)

            # 比较temp_structure与aligned_combined_pdb_files.pdb中所有结构的RMSD值
            should_keep = True
            for aligned_structure, aligned_pdb_file in zip(aligned_structures, pdb_files):
                rmsd = align_and_calculate_rmsd(aligned_structure, temp_structure, aligned_pdb_file)
                print(f"RMSD between {os.path.basename(pdb_file)} and {os.path.basename(aligned_pdb_file)}: {rmsd}")
                if rmsd < rmsd_threshold:
                    should_keep = False
                    break
            
            # 如果不存在RMSD小于阈值的结果，则将temp_structure写入最终的合并文件
            if should_keep:
                with open(pdb_file, 'r') as temp_file:
                    lines = temp_file.readlines()
                    for line in lines:
                        if line.startswith(('REMARK', 'ATOM', 'HETATM', 'END')):
                            aligned_combined_pdb.write(line)
                
                conformer_count += 1
                aligned_structures.append(temp_structure)

    # 输出实际生成的构象个数
    print(f"实际生成的构象个数为: {conformer_count}")

# 从combined_sorted_pdb_files.pdb文件中读取所有PDB结构并进行RMSD筛选
def screen_rmsd(rmsd_threshold):
    parser = PDBParser(QUIET=True)

    # 读取combined_pdb_files.pdb文件
    with open('combined_sorted_pdb_files.pdb', 'r') as f:
        content = f.read().strip().split('END\n')

    # 将每个PDB结构写入单独的文件
    pdb_files = []
    for i, pdb_content in enumerate(content):
        if pdb_content.strip():
            pdb_filename = f'pdb_{i}.pdb'
            with open(pdb_filename, 'w') as pdb_file:
                pdb_file.write(pdb_content + 'END\n')
            pdb_files.append(pdb_filename)

    # 创建一个新的文件来存储所有对齐后的PDB结构
    aligned_combined_pdb_filename = 'aligned_combined_pdb_files.pdb'

    align_structures_and_check_rmsd(pdb_files, parser, aligned_combined_pdb_filename, rmsd_threshold)

    # 删除中间生成的PDB文件
    for pdb_file in pdb_files:
        os.remove(pdb_file)

    print(f"已创建合并PDB文件: aligned_combined_pdb_files.pdb")

try:
    screen_rmsd(rmsd_threshold)

except Exception as e:
    print(f"错误: {e}")

1.8
RMSD between pdb_1.pdb and pdb_0.pdb: 0.17983554003210547
RMSD between pdb_2.pdb and pdb_0.pdb: 0.17983554003210547
RMSD between pdb_3.pdb and pdb_0.pdb: 0.25709062273838185
RMSD between pdb_4.pdb and pdb_0.pdb: 0.06210736555629287
RMSD between pdb_5.pdb and pdb_0.pdb: 0.05707898156198986
RMSD between pdb_6.pdb and pdb_0.pdb: 0.21408144714537145
RMSD between pdb_7.pdb and pdb_0.pdb: 0.07829706915694078
RMSD between pdb_8.pdb and pdb_0.pdb: 0.09825986776426197
RMSD between pdb_9.pdb and pdb_0.pdb: 0.1944415798097
RMSD between pdb_10.pdb and pdb_0.pdb: 0.27922900689288804
RMSD between pdb_11.pdb and pdb_0.pdb: 0.1986055337772643
RMSD between pdb_12.pdb and pdb_0.pdb: 0.1986055337772643
RMSD between pdb_13.pdb and pdb_0.pdb: 0.3206170066769032
RMSD between pdb_14.pdb and pdb_0.pdb: 0.22939494327891363
RMSD between pdb_15.pdb and pdb_0.pdb: 1.7477964409871318
RMSD between pdb_16.pdb and pdb_0.pdb: 0.19461203247054895
RMSD between pdb_17.pdb and pdb_0.pdb: 0.13465169563976687
RMSD betwe

In [99]:
# 定义split_pdb_directory文件夹
split_pdb_directory = 'split_pdbs'

# 从PDB文件中删除包含'ACE'或'NME'的行，并重新编写原子编号
def remove_ace_nme_lines(pdb_filename):

    lines = []
    current_residue_number = 1

    with open(pdb_filename, 'r') as f:
        for line in f:
            if 'ACE' in line or 'NME' in line:
                continue
            if line.startswith('ATOM'):
                # 重新编写原子编号
                line = f"{line[:6]}{current_residue_number:>4}{line[10:]}"
                current_residue_number += 1
            lines.append(line)

    # 将修改后的行写回原始PDB文件
    with open(pdb_filename, 'w') as f:
        f.write(''.join(lines))

# 将PDB文件拆分为单个构象
def split_pdb_file(input_pdb_file):

    output_directory = 'split_pdbs'
    os.makedirs(output_directory, exist_ok=True)

    conformation_index = 1
    pdb_content = []

    with open(input_pdb_file, 'r') as f:
        for line in f:
            if line.strip() == 'END':
                pdb_filename = os.path.join(output_directory, f'conformation_{conformation_index}.pdb')
                with open(pdb_filename, 'w') as pdb_file:
                    pdb_file.write('\n'.join(pdb_content) + '\nENDMDL\n')

                conformation_index += 1
                pdb_content = []
            else:
                pdb_content.append(line.strip())

    if pdb_content:
        pdb_filename = os.path.join(output_directory, f'conformation_{conformation_index}.pdb')
        with open(pdb_filename, 'w') as pdb_file:
            pdb_file.write('\n'.join(pdb_content) + '\nENDMDL\n')

# 读取XXX_temps.params文件并替换PDB文件中的[12:16]为非天然氨基酸名称
def process_all_pdb_files(directory, non_natural_aa_codes):

    for filename in os.listdir(directory):
        if filename.endswith('.pdb'):
            pdb_filepath = os.path.join(directory, filename)
            remove_ace_nme_lines(pdb_filepath)
            # 读取PDB文件并替换[12:16]为非天然氨基酸名称
            lines = []
            with open(pdb_filepath, 'r') as f:
                for idx, line in enumerate(f):
                    if line.strip() == 'ENDMDL':
                        lines.append(line)
                        continue
                    if idx < len(non_natural_aa_codes):
                        code = non_natural_aa_codes[idx]
                        line = line[:12] + code + line[16:]
                    lines.append(line)
            
            # 将修改后的行写回原始PDB文件
            with open(pdb_filepath, 'w') as f:
                f.write(''.join(lines))

            print(f"Processed: {filename}")

# 将split_pdb_directory中的所有PDB文件合并为单个输出文件
def combine_pdb_files(output_filename):

    with open(output_filename, 'w') as output_file:
        conformation_index = 1
        for filename in sorted(os.listdir(split_pdb_directory)):
            if filename.endswith('.pdb'):
                pdb_filepath = os.path.join(split_pdb_directory, filename)
                with open(pdb_filepath, 'r') as pdb_file:
                    output_file.write(f"MODEL     {conformation_index}\n")
                    for line in pdb_file:
                        output_file.write(line)
                    conformation_index += 1

# 删除split_pdb_directory
def remove_split_pdb_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

    try:
        os.rmdir(directory)
    except Exception as e:
        print(f"Failed to delete {directory}. Reason: {e}")

# 调整索引号，将第二列（索引10:11）减去12，使其从1开始编号
def adjust_hetatm_records(pdb_filename):

    lines = []
    with open(pdb_filename, 'r') as f:
        for line in f:
            if line.startswith('HETATM'):
                original_number = int(line[9:11])
                modified_number = original_number - 12
                # Adjust right alignment
                if modified_number < 10:
                    line = f"{line[:9]} {modified_number}{line[11:]}"
                else:
                    line = f"{line[:9]}{modified_number}{line[11:]}"
            lines.append(line)

    with open(pdb_filename, 'w') as f:
        f.writelines(lines)

# 执行函数，调用以上modify功能，对pdb文件进行修改
def modify(non_natural_aa_abbreviation):

    params_filename = f"{non_natural_aa_abbreviation}_temps.params"

    # 读取XXX_temps.params文件中的非天然氨基酸原子编号
    non_natural_aa_codes = []
    with open(params_filename, 'r') as params_file:
        for line in params_file:
            if line.startswith('ATOM'):
                code = line.strip()[5:9]
                non_natural_aa_codes.append(code)

    # 拆分合并的PDB文件
    input_pdb_file = 'aligned_combined_pdb_files.pdb'
    split_pdb_file(input_pdb_file)

    # 处理拆分后的PDB文件
    process_all_pdb_files(split_pdb_directory, non_natural_aa_codes)

    # 合并处理后的PDB文件
    combined_pdb_filename = 'merged_combined_pdb_files.pdb'
    combine_pdb_files(combined_pdb_filename)

    # 调整原子索引号
    adjust_hetatm_records(combined_pdb_filename)

    # 删除split_pdb_directory
    remove_split_pdb_directory(split_pdb_directory)
    print(f"Combined PDB files into '{combined_pdb_filename}' and deleted '{split_pdb_directory}'.")

non_natural_aa_abbreviation = n_value

try:
    modify(non_natural_aa_abbreviation)
except Exception as e:
    print(f"Error: {e}")

Processed: conformation_8.pdb
Processed: conformation_3.pdb
Processed: conformation_7.pdb
Processed: conformation_4.pdb
Processed: conformation_9.pdb
Processed: conformation_10.pdb
Processed: conformation_1.pdb
Processed: conformation_5.pdb
Processed: conformation_6.pdb
Processed: conformation_2.pdb
Combined PDB files into 'merged_combined_pdb_files.pdb' and deleted 'split_pdbs'.


In [100]:
# 删除临时文件
def delete_files():
    files_to_delete = ['combined_pdb_files.pdb', 'combined_sorted_pdb_files.pdb', 'aligned_combined_pdb_files.pdb']
    
    for file in files_to_delete:
        try:
            os.remove(file)
            print(f"Successfully deleted {file}")
        except FileNotFoundError:
            print(f"File {file} not found")
        except PermissionError:
            print(f"Permission denied to delete {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")

    if os.path.exists('pdb_files'):
        shutil.rmtree('pdb_files')
        print("Deleted 'pdb_files' directory")

    if os.path.exists('split_pdb_files'):
        shutil.rmtree('split_pdb_files')
        print("Deleted 'split_pdb_files' directory")

delete_files()

Successfully deleted combined_pdb_files.pdb
Successfully deleted combined_sorted_pdb_files.pdb
Successfully deleted aligned_combined_pdb_files.pdb
Deleted 'pdb_files' directory
Deleted 'split_pdb_files' directory
